# 🏦 Loan Default Risk Prediction - Financial Risk Analytics

This notebook implements a complete machine learning pipeline for predicting loan default risk. It includes data preprocessing, hyperparameter tuning using **GridSearchCV**, and comparative analysis between **Logistic Regression** and **SVM**.

### Algorithms & Optimization:
- **Logistic Regression**: Regularization tuning (C, penalty).
- **SVM**: Kernel comparison (linear, rbf, poly), C, and Gamma tuning.
- **Evaluation**: Accuracy, ROC-AUC, Confusion Matrix.
- **Cross-Validation**: 5-fold CV for robust results.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully.")

### 1. Load Data
Please make sure `train.csv` and `test_Y3wMUE5_7gLdaTN.csv` are uploaded to your Colab session.

In [ ]:
try:
    train_df = pd.read_csv('train.csv')
    # Note: Use the specific name provided by the user
    test_df = pd.read_csv('test_Y3wMUE5_7gLdaTN.csv')
    print("✅ Data loaded successfully.")
except FileNotFoundError:
    print("❌ Files not found. Please upload 'train.csv' and 'test_Y3wMUE5_7gLdaTN.csv' using the Files tab on the left.")

### 2. Data Cleaning & Preprocessing
We fill missing values (Mode for categorical, Median for numerical) and encode categorical features.

In [ ]:
def preprocess_data(df, is_train=True):
    df_copy = df.copy()
    if 'Loan_ID' in df_copy.columns: df_copy.drop('Loan_ID', axis=1, inplace=True)
    
    cat_cols = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Credit_History', 'Property_Area']
    for col in cat_cols: 
        if col in df_copy.columns: df_copy[col].fillna(df_copy[col].mode()[0], inplace=True)
            
    num_cols = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term']
    for col in num_cols: 
        if col in df_copy.columns: df_copy[col].fillna(df_copy[col].median(), inplace=True)
    
    mapping = {
        'Gender': {'Male': 1, 'Female': 0},
        'Married': {'Yes': 1, 'No': 0},
        'Education': {'Graduate': 1, 'Not Graduate': 0},
        'Self_Employed': {'Yes': 1, 'No': 0},
        'Property_Area': {'Urban': 2, 'Semiurban': 1, 'Rural': 0},
        'Dependents': {'0': 0, '1': 1, '2': 2, '3+': 3},
        'Loan_Status': {'Y': 1, 'N': 0}
    }
    
    for col, map_val in mapping.items():
        if col in df_copy.columns: df_copy[col] = df_copy[col].map(map_val)
    return df_copy

train_cleaned = preprocess_data(train_df)
test_cleaned = preprocess_data(test_df)

X = train_cleaned.drop('Loan_Status', axis=1)
y = train_cleaned['Loan_Status']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(test_cleaned) if not test_cleaned.empty else None

print("✅ Preprocessing complete.")

### 3. Logistic Regression Optimization
Tuning for $C$ and penalty using GridSearchCV.

In [ ]:
lr_param_grid = {'C': [0.01, 0.1, 1, 10, 100], 'penalty': ['l1', 'l2'], 'solver': ['liblinear']}
lr_grid = GridSearchCV(LogisticRegression(random_state=42), lr_param_grid, cv=5, scoring='accuracy')
lr_grid.fit(X_train_scaled, y_train)
best_lr = lr_grid.best_estimator_
print(f"Best Logistic Regression Params: {lr_grid.best_params_}")

### 4. SVM Optimization
Tuning Kernel, $C$, and Gamma.

In [ ]:
svm_param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf', 'poly'], 'gamma': ['scale', 'auto']}
svm_grid = GridSearchCV(SVC(probability=True, random_state=42), svm_param_grid, cv=5, scoring='accuracy')
svm_grid.fit(X_train_scaled, y_train)
best_svm = svm_grid.best_estimator_
print(f"Best SVM Params: {svm_grid.best_params_}")

### 5. Evaluation & Comparison

In [ ]:
def evaluate(model, X_v, y_v, name):
    preds = model.predict(X_v)
    acc = accuracy_score(y_v, preds)
    auc = roc_auc_score(y_v, model.predict_proba(X_v)[:, 1])
    print(f"\n{name} Results:")
    print(f"Accuracy: {acc:.4f} | ROC-AUC: {auc:.4f}")
    return acc, auc

lr_res = evaluate(best_lr, X_val_scaled, y_val, "Logistic Regression")
svm_res = evaluate(best_svm, X_val_scaled, y_val, "SVM")

# Viz
plt.figure(figsize=(10, 4))
sns.barplot(x=['Logistic Regression', 'SVM'], y=[lr_res[0], svm_res[0]], palette='viridis')
plt.title('Accuracy Comparison')
plt.ylim(0, 1)
plt.show()

### 6. Predict on Test Data

In [ ]:
best_overall = best_svm if svm_res[0] > lr_res[0] else best_lr
if X_test_scaled is not None:
    test_preds = best_overall.predict(X_test_scaled)
    test_df['Loan_Status_Predicted'] = ['Y' if p == 1 else 'N' for p in test_preds]
    test_df.to_csv('loan_predictions_result.csv', index=False)
    print("✅ Predicted results saved to 'loan_predictions_result.csv'")
    display(test_df[['Loan_ID', 'Loan_Status_Predicted']].head())